Autor: Daniel Flores

# Librerías

In [1]:
!pip -q install langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.4 MB/s eta 0:00:00


In [2]:
!pip -q install langchain-huggingface

In [3]:
!pip -q install langchain-pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 4.3 MB/s eta 0:00:00


In [4]:
!pip install -qU langchain_community pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.3 MB/s eta 0:00:00


In [5]:
!pip -q install langchain-community

# Credenciales

In [6]:
import os
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GEN_TUTOR')
# Establecer la clave de API en las variables de entorno
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [7]:
import getpass
import os
import time

from pinecone import Pinecone, ServerlessSpec

from google.colab import userdata
PINECONE_API_KEY=userdata.get('Pinecone')

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

pinecone_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api_key)

# Embedding Model


In [8]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddingsHF = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/402 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddingsGAI = GoogleGenerativeAIEmbeddings(model="text-embedding-004")

# Vector Store

In [ ]:
import time

index_name = "quickstart"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

index = pc.Index(index_name)

In [ ]:
index

In [ ]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddingsHF)

# Lectura de Datos

## Limpieza V1

In [ ]:
import re

def clean_text(text):
    # Eliminar números de página y saltos de línea extras
    text = re.sub(r'\d+\s*\n\s*', '', text)

    # Eliminar caracteres especiales y espacios extras
    text = re.sub(r'\s+', ' ', text).strip()

    # Eliminar referencias entre paréntesis
    text = re.sub(r'\s*\([^)]*\)', '', text)

    # Eliminar códigos de gradación
    text = re.sub(r'E-\s*[A-Za-z0-9]+\s*R-\s*[A-Za-z0-9]+', '', text)

    # Eliminar encabezados y títulos
    text = re.sub(r'^(.+?\n)+', '', text)

    return text

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

# Cargar múltiples PDFs individualmente
loader1 = PyPDFLoader("/content/drive/MyDrive/UTPL/CursoIntroMad/Guía didáctica_Curso Introducción.pdf")
loader2 = PyPDFLoader("/content/drive/MyDrive/UTPL/CursoIntroMad/Plan_docente_Curso introductorio-oct2024 -febrero 2025 autorizado.pdf")

guia_didactica = loader1.load()
plan_docente = loader2.load()


In [ ]:
guia_didactica = guia_didactica[5:] #Eliminar páginas de índice y portada

In [ ]:
len(guia_didactica)

51

In [ ]:
# Combinar documentos
docsMAD = guia_didactica + plan_docente

In [ ]:
len(docsMAD)

64

In [ ]:
(docsMAD[51])

Document(metadata={'source': '/content/drive/MyDrive/UTPL/CursoIntroMad/Plan_docente_Curso introductorio-oct2024 -febrero 2025 autorizado.pdf', 'page': 0}, page_content=' \n \n \n \n \n \n \nUNIVERSIDAD TÉCNICA PARTICULAR DE LOJA \nVICERRECTORADO DE MODALIDAD ABIERTA Y A DISTANCIA \n \nFACULTAD:  Todas \nCARRERA: Todas \n \n \n \nPLAN DOCENTE DEL CURSO INTRODUCCIÓN A LA \nMODALIDAD A DISTANCIA Y EN LÍNEA (PROPEDÉUTICO) \n \n \nDiseñado por: Mariana Buele Maldonado \nRita Cabrera González \nMayra Eugenia Armijos \n \nMODALIDAD DE ESTUDIO: A DISTANCIA Y EN LÍNEA \n \n \n                  PERÍODO ACADÉMICO \nOCTUBRE 2024 - FEBRERO 2025\n')

In [ ]:
from langchain_core.documents import Document
# Aplicar la función de limpieza manteniendo la estructura de Document
docsMAD_clean = [
    Document(
        page_content=clean_text(doc.page_content),
        metadata=doc.metadata
    ) for doc in docsMAD
]

In [ ]:
len(docsMAD_clean)

64

In [ ]:
docsMAD_clean[51]

Document(metadata={'source': '/content/drive/MyDrive/UTPL/CursoIntroMad/Guía didáctica_Curso Introducción.pdf', 'page': 5}, page_content='2. Metodología de aprendizaje Con el objetivo de aportar a los resultados de aprendizaje del curso Introducción a la Modalidad a distancia y en línea, se aplicará un proceso metodológico de aprendizaje activo, que valora todo el proceso de conocimiento y experiencias que trae consigo el nuevo estudiante desde el bachillerato. Aprendizaje significativo Aprender desde las emociones a partir de nuestras experiencias, nos llevan al saber pensar y hacer. El aprendizaje activo se iniciará desde su propia experiencia para facilitar el análisis y la reflexión que aporta al desarrollo del pensamiento crítico, de la observación, de la identificación de las ideas principales que faciliten poner UTPL 6')

### Carga en la base de datos Pinecone

In [ ]:
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(docsMAD_clean))]

#Esta línea envia los documentos con sus nuevos ids a la base
vector_store.add_documents(documents=docsMAD_clean, ids=uuids)

['67fb1c33-014f-47a1-bb64-1f8a275bc5ed',
 'e406055b-8e76-4fef-bbf1-b01538c214c6',
 '0cfa5236-d0c5-4832-a529-1b6928ad6e48',
 'a80ed7a6-e727-411d-a3a0-91d1ee92c531',
 'f84c906e-79b7-486a-a72c-2d1b2c61ff55',
 'b94fca84-fba4-4679-9042-1fa6c69c51e6',
 'd911866d-25b9-4ef4-bf85-6966ec4c7029',
 'ced20a1f-76aa-48b0-8d4c-b2964e3bf1ad',
 '2e53d5c3-4f95-40e0-9bc3-0f0f27f22fa6',
 '7390c8b7-bbda-43a9-8b2f-0ac94e75cfe2',
 'd9366b25-215d-4674-9cfe-84e5a7acaa03',
 'be0ae25f-febd-4320-aa36-bdf82137769e',
 'ff4308fb-7420-4723-a749-bb7fcd6e3bf7',
 'd82d5317-0235-43c2-bdc9-9f33b7a8b87c',
 '1edf64e1-12c9-4b2a-bd8b-176fb207c0e7',
 'd708140d-71f3-41aa-8cf6-91c67e61af9e',
 'd81a74f9-0065-4a44-9f2a-3ec75d1e6152',
 'e0b64094-cc7c-429b-ab67-e1c1acb2167d',
 'd5819f09-8fc6-4d3a-8e5a-94d525302c0b',
 'b2fb78f4-04f7-4c13-af60-a7f68a587d98',
 '94a38e61-4bc7-4039-93fc-6f477914cd38',
 '50a6df1d-0c1b-49f6-a1e0-64466d0aa2f4',
 '5a874712-066b-4722-9176-d6f3787db533',
 'c669f977-e324-48fd-9ca8-b022f6e3a0e8',
 '584225b0-4fcc-

In [ ]:
results = vector_store.similarity_search(
    "El aprendizaje activo se iniciará desde su propia experiencia para facilitar el análisis y la reflexión",
    k=2
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* 2. Metodología de aprendizaje Con el objetivo de aportar a los resultados de aprendizaje del curso Introducción a la Modalidad a distancia y en línea, se aplicará un proceso metodológico de aprendizaje activo, que valora todo el proceso de conocimiento y experiencias que trae consigo el nuevo estudiante desde el bachillerato. Aprendizaje significativo Aprender desde las emociones a partir de nuestras experiencias, nos llevan al saber pensar y hacer. El aprendizaje activo se iniciará desde su propia experiencia para facilitar el análisis y la reflexión que aporta al desarrollo del pensamiento crítico, de la observación, de la identificación de las ideas principales que faciliten poner UTPL 6 [{'page': 5.0, 'source': '/content/drive/MyDrive/UTPL/CursoIntroMad/Guía didáctica_Curso Introducción.pdf'}]
* 2. ¿Qué aplicaciones tiene el pensamiento analítico en la vida de un estudiante? 3. ¿Cómo resolver un ejercicio de pensamiento analítico? 2.4 Pensamiento Abstracto 1. ¿Cómo define uste

# RAG


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-8b",google_api_key=GOOGLE_API_KEY, temperature=0.3, max_tokens=500)

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def rag(pregunta):
  retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 4, "score_threshold": 0.5},
  )

  prompt_template = """
    Actúa como un tutor virtual especializado en brindar explicaciones claras y detalladas.
    Responde las siguientes preguntas basándote únicamente en el siguiente contexto,
    proporcionando ejemplos prácticos y explicaciones paso a paso cuando sea necesario:
    {context}
    Pregunta: {question}
  """
  prompt = ChatPromptTemplate.from_template(prompt_template)

  chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
  )
  result = chain.invoke(pregunta)
  print(result)


Conteo de busquedas: 12


# Preguntas al RAG

## Sin límite de tokens y con Flash 1.5:

In [ ]:
rag("como uso el EVA?")

El texto describe el EVA como un espacio de interacción para estudiantes, accesible desde una aplicación móvil (Canvas Student) y a través de un navegador web.  Su uso abarca diversas funciones:

**1. Comunicación:**

* **Bandeja de entrada:** Permite mantenerse comunicado desde cualquier lugar.

**2. Aprendizaje:**

* **Autoevaluaciones:**  Se ofrecen autoevaluaciones para verificar el conocimiento sobre temas específicos (ej: verdadero/falso sobre el tema 1.1).  Estas proporcionan retroalimentación para reforzar el aprendizaje.
* **Actividades del curso:** El EVA contiene diferentes tipos de actividades para verificar el logro de los resultados de aprendizaje, incluyendo:
    * **Tareas:**  Envío de trabajos.
    * **Evaluaciones:** Pruebas o exámenes.
    * **Foros de discusión:** Participación en debates.
    * **Chat y videocolaboración:**  Interacción en tiempo real con otros estudiantes o profesores.  Cada actividad tiene parámetros como calificación, fechas límite, duración e i

## Sin limite de tokens con Flash 8B

In [ ]:
#Sin limite de tokens pero con el flash 8B
rag("Como uso el EVA?")

El EVA (Entorno Virtual de Aprendizaje) te permite interactuar y acceder a recursos de aprendizaje.  Para utilizarlo correctamente, debes seguir estos pasos y considerar las diferentes herramientas:

**1. Navegación:**

* **Menú de navegación global:**  Te permite acceder a diferentes secciones del curso, como contenido, herramientas y navegación del curso.  Observa la imagen 1, 2, 3 y 4 (si las imágenes están disponibles) para identificar las opciones en el menú.
* **Bandeja de entrada:**  Mantente comunicado con la plataforma desde cualquier lugar.  Utiliza esta función para interactuar con tus compañeros y tutores.

**2. Actividades de aprendizaje:**

* **Autoevaluaciones:**  El EVA ofrece autoevaluaciones para que evalúes tus conocimientos.  Un ejemplo es la autoevaluación sobre los temas 1 al 4, que se basa en preguntas de verdadero o falso sobre el tema 1.1.  Cada respuesta te proporciona retroalimentación.
* **Tareas, evaluaciones y foros:**  Estas son actividades comunes en el 

## Con límite de tokens (500) y flash 1.5:

In [ ]:
rag("como uso el EVA?")

El texto describe el EVA como un espacio de interacción en línea, accesible también a través de una aplicación móvil llamada Canvas Student.  Su uso abarca varias funciones clave:

**1. Comunicación:**  La bandeja de entrada del EVA permite mantenerse comunicado desde cualquier lugar.

**2. Aprendizaje:** El EVA contiene:

* **Contenidos del curso:**  Material de aprendizaje organizado en unidades y temas.
* **Actividades de aprendizaje:**  Para verificar el logro de los resultados de aprendizaje.  Estas incluyen:
    * **Tareas:**  Envío de trabajos.
    * **Evaluaciones:**  Pruebas o exámenes.
    * **Foros de discusión:**  Espacios para interactuar con otros estudiantes y el profesor.
    * **Chat y videocolaboración:**  Herramientas para comunicación en tiempo real.  Cada actividad tiene parámetros como calificación, fechas límite, duración e intentos permitidos. Es crucial prestar atención a esta parametrización.
* **Autoevaluaciones:**  Para comprobar el conocimiento adquirido so

In [ ]:
rag("Dime 3 preguntas para comprobar mi entendimiento del EVA")

Basándome en los documentos proporcionados, aquí hay 3 preguntas para comprobar tu entendimiento del EVA (Entorno Virtual de Aprendizaje), junto con las respuestas esperadas y explicaciones:


**Pregunta 1:** ¿De qué maneras puedo ordenar la lista de calificaciones en el EVA?

**Respuesta esperada:**  La lista de calificaciones se puede ordenar por fecha de entrega, grupo de tareas, módulo y título de la actividad. La fecha de entrega es la opción predeterminada.

**Explicación:**  El documento menciona explícitamente estas opciones de ordenamiento en la sección de calificaciones.  Esto permite al estudiante una mayor flexibilidad para revisar su progreso en el curso.


**Pregunta 2:** ¿Qué tipos de actividades de aprendizaje puedo encontrar en el EVA, y qué información importante debo considerar al participar en ellas?

**Respuesta esperada:** El EVA incluye tareas, evaluaciones, foros de discusión, chat y videocolaboración.  Al participar, es crucial considerar la parametrización de 

## Con límite de tokens (500) y flash 8B:

In [ ]:
rag("como uso el EVA?")

El documento describe cómo usar el Entorno Virtual de Aprendizaje (EVA) de la UTPL.  No hay una única respuesta a "cómo usar el EVA", ya que depende de la actividad específica que se quiera realizar.  Sin embargo, el documento proporciona información sobre varias funcionalidades y recursos.

Aquí te presento una guía paso a paso sobre cómo usar algunas de las funcionalidades más comunes:

**1. Navegación y Comunicación:**

* **Bandeja de entrada:** El EVA te permite mantenerte comunicado desde cualquier lugar.  Para usarla, debes acceder a la bandeja de entrada del EVA.  No se detalla cómo acceder a ella, pero se asume que es parte de la interfaz principal.
* **Menú de navegación global, menú de herramientas del curso, menú de navegación del curso:**  Estos menús te guían por el contenido del curso, las herramientas disponibles y la estructura del EVA.  Para usarlos, navega por las opciones disponibles en la interfaz principal.

**2. Actividades de Aprendizaje:**

* **Autoevaluaciones:

In [ ]:
rag("Dime 3 preguntas para comprobar mi entendimiento del EVA")

Basándome en el texto proporcionado, aquí tienes 3 preguntas para comprobar tu entendimiento del Entorno Virtual de Aprendizaje (EVA):

1. **¿Qué opciones de ordenamiento ofrece la sección de "Calificaciones" del EVA, y cuál es la predeterminada?**  (Esta pregunta evalúa tu comprensión de la funcionalidad de la sección de calificaciones dentro del EVA, incluyendo la capacidad de ordenar por diferentes criterios y el conocimiento del criterio predeterminado).


2. **Describe tres tipos de actividades de aprendizaje que se pueden encontrar en un EVA, y explica brevemente cómo se diferencian entre sí.** (Esta pregunta evalúa tu comprensión de la variedad de actividades que un EVA puede ofrecer, incluyendo tareas, evaluaciones y foros de discusión, y tu capacidad para distinguirlas).


3. **¿Cómo se pueden gestionar las notificaciones en el EVA y qué información adicional se puede registrar en la sección de "Canales de contacto"?** (Esta pregunta evalúa tu comprensión de la configuración p

## Otras Preguntas

In [ ]:
rag("que debo hacer en la semana 2")

Basándome únicamente en los documentos proporcionados, no hay información específica sobre qué se debe hacer en la semana 2.  Los documentos describen aspectos del entorno virtual de aprendizaje (EVA) de la UTPL, incluyendo el uso del calendario, la bandeja de entrada, la gestión de archivos y la autoevaluación de habilidades (mencionada para la semana 4).  No hay un plan de estudios o calendario detallado que indique las tareas para la semana 2.



In [ ]:
rag("qué permite la herramienta studio?")

Basado en el texto proporcionado, la herramienta Studio permite grabar, cargar y compartir videos.  También permite agregar videos desde un computador y desde YouTube.



In [ ]:
rag("En que semana es la Autoevaluación de habilidades básicas y compromiso de mejora para el estudio en la carrera?")

La autoevaluación de habilidades básicas y compromiso de mejora para el estudio en la carrera se realiza en la cuarta semana.



In [ ]:
rag("que debo realizar en la semana 4")

La información proporcionada no especifica qué actividades se deben realizar en la semana 4.  Los documentos describen las funcionalidades de la plataforma UTPL (Calendario, Bandeja de entrada, Archivos, Perfil, etc.), pero no incluyen un plan de estudios o un calendario con las tareas semanales.  Para saber qué hacer en la semana 4, necesitaría consultar el programa del curso o el calendario académico proporcionado por su instructor.



In [ ]:
rag("de que trata la guía didáctica?")

Basándome únicamente en los documentos proporcionados, la guía didáctica del curso "Introducción a la Modalidad a Distancia y en Línea" trata sobre la familiarización con el Entorno Virtual de Aprendizaje (EVA) y el sistema de educación a distancia de la universidad.  

Específicamente, se enfoca en:

* **Familiarización con el EVA (Unidad 1):**  El documento menciona un video introductorio e indicaciones generales para navegar y utilizar el EVA.  El objetivo es que el estudiante se familiarice con las funcionalidades del EVA para desarrollar las actividades del curso.  Se evalúa mediante un cuestionario de 10 puntos.

* **Sistema de Educación a Distancia (Unidad 2):**  Esta unidad cubre el modelo educativo y pedagógico de la modalidad a distancia y en línea, además de adquirir competencias relacionadas con hábitos y técnicas de estudio.  También se evalúa con un cuestionario de 10 puntos.

* **Fase Presencial:**  La guía menciona talleres presenciales en centros de apoyo al final del 

In [ ]:
rag("")